In [1]:
import sqlite3


In [2]:
import importlib

import pandas as pd
import optuna
import gc
import copy

import sys
sys.path.append("../nlp_assist/")

In [3]:
import datetime
import pickle
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
ts = now.strftime('%Y%m%d_%H%M%S_%f')
ts

'20230216_183200_750595'

In [4]:
#カラム内の文字数。デフォルトは50だった
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", 101)

In [5]:
#条件もとの3テーブルを作るモジュール
import nlp_connect as nl
importlib.reload(nl)

#各種条件をquboとして作るためのモジュール
import nlp_limiter as lmt
importlib.reload(lmt)

#並列で探索するソルバのラッパー
import p_solver as ps
importlib.reload(ps)

#結果チェッカ等
import res_parser as rps
importlib.reload(rps)

import nsp_settings as ns
importlib.reload(ns)

print("reloaded!")
f = "../demos/dmk0.3_20220111_18.xlsx"
f2 = "../demos/dmk0.3_20220111_18add.xlsx"

x = nl.xlsx2condtion(f)
x.make3CDT(year=2020, month= 11)

# q_genはqubo generator
qg = lmt.q_gen(x2c=x)

dummy var generator activated
start status is 0
limter activated!
dummy var generator activated
start status is 0
limter activated!
reloaded!
xlsx file is
../demos/dmk0.3_20220111_18.xlsx
make condition tables.
holidaymode: asHoliday
year:  2020
month:  11
holidaymode:asHoliday
days that is holiday and not sunday will be Holiday.
Q generator activated!
x2c need to pre make 3DT
nsp_expression activated!


In [6]:
f_df = "./optuna_study_itr_whole_fix_lchs.db"

con = sqlite3.connect(f_df)

In [7]:
def q2df(q):
    df = pd.DataFrame(con.execute(q))
    
    len_col = len(df.columns)
    df.columns = ["c"+str(i+1) for i in range(len_col)]
    return df

In [8]:
df_t = q2df("select name from sqlite_master where type='table';")

In [9]:
df_t

,c1
0,studies
1,version_info
2,study_directions
3,study_user_attributes
4,study_system_attributes
5,trials
6,trial_user_attributes
7,trial_system_attributes
8,trial_params
9,trial_values


In [10]:
df_trial = q2df("select * from trial_values")\
    .query("c4 == 1")
df_trial


,c1,c2,c3,c4,c5
770,771,775,0,1.0,FINITE
818,819,825,0,1.0,FINITE
851,852,857,0,1.0,FINITE
941,942,949,0,1.0,FINITE
1087,1088,1090,0,1.0,FINITE
1115,1116,1120,0,1.0,FINITE
1124,1125,1133,0,1.0,FINITE
1195,1196,1197,0,1.0,FINITE
1228,1229,1233,0,1.0,FINITE
1230,1231,1238,0,1.0,FINITE


In [11]:
tirals_one = df_trial["c2"]

In [12]:
df_param = q2df("select * from trial_params") \
    .query("c2 in @tirals_one")

# .query()
df_param.head()

,c1,c2,c3,c4,c5
13932,13933,775,1by1,146.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"
13933,13934,775,cnight,101.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"
13934,13935,775,cwork,5.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"
13935,13936,775,dab,75.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"
13936,13937,775,fseq,187.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"


In [13]:
df_param_sub = df_param \
    .loc[:, ["c2", "c3", "c4"]]
# d_params.head().to_dict()
d_params = dict()
for r in df_param_sub.itertuples():
    if r.c2 not in d_params.keys():
        d_params[r.c2] = dict()
    d_params[r.c2][r.c3] = r.c4

In [14]:
d_params[775]

{'1by1': 146.0,
 'cnight': 101.0,
 'cwork': 5.0,
 'dab': 75.0,
 'fseq': 187.0,
 'lcnn': 92.0,
 'lcnt': 103.0,
 'lfbd': 146.0,
 'lnd': 168.0,
 'mps': 122.0,
 'nnv': 105.0,
 'nseq': 145.0,
 'rqh': 14.0,
 'rqn': 183.0,
 'sc': 64.0,
 'tn': 1.0,
 'tt': 23.0,
 'sweeps': 473.0}

In [15]:
import importlib

import pandas as pd
import optuna
import gc
import copy

import sys
sys.path.append("../nlp_assist/")

import pandas as pd
import optuna
import gc
import copy

#条件もとの3テーブルを作るモジュール
import nlp_connect as nl
importlib.reload(nl)

#各種条件をquboとして作るためのモジュール
import nlp_limiter as lmt
importlib.reload(lmt)

#並列で探索するソルバのラッパー
import p_solver as ps
importlib.reload(ps)

#結果チェッカ等
import res_parser as rps
importlib.reload(rps)

import nsp_settings as ns
importlib.reload(ns)

print("reloaded!")


dummy var generator activated
start status is 0
limter activated!
reloaded!


In [16]:
f = "../demos/dmk0.3_20220111_18.xlsx"
f2 = "../demos/dmk0.3_20220111_18add.xlsx"
# x = nl.xlsx2condtion(f)

In [17]:
inc = ('1by1','cnight',
     'cwork',
     'dab',
    #  'eqg2',
    #  'eqg3',
     'fseq',
     'lcnn',
     'lcnt',
     'lfbd',
     'lnd',
     'mps',
     'nnv',
     'nseq',
     'rqh',
     'rqn',
     'sc',
     'tn',
     'tt')

In [18]:
res_dict = dict()

In [ ]:
%%time
for k, v in d_params.items():
    print(k)
    swmul = v["sweeps"]
    param_dict = v.copy()
    del param_dict["sweeps"]
    sw_base = 1e+4
    # sw_base = 1e+5
    sw = int(sw_base *swmul * 4)
    x = nl.xlsx2condtion(f)
    x.make3CDT()

    # q_genはqubo generator
    qg = lmt.q_gen(x2c=x)
    qg.setParam(param_dict)
    print(param_dict)
    qg.makeAllLimitation(do_normalize=True, hideProgressBar=True, include=inc)
    
    p = ps.psol(q=qg.ne.getQ(),
                 num_reads=1,
                 sweeps=sw)
    mins = p.p_solve_getMin(threads=15, times=15)

    rp = rps.res_parser(x=x)
    rp.prepRes(res=mins.sample, hideProgressBar=True)
    rc = rps.res_checker(x=x, echo_bool=True)
    rc.checkAllLimitation(res_parser=rp, include=inc)
    
    tm = rps.table_maker(x=x, res=mins.sample)
    

    res_dict[k] = dict()
    res_dict[k]["errornum"] = len(rc.log_df)
    res_dict[k]["mins_sample"] = mins.sample
    res_dict[k]["log_df"] = rc.log_df
    res_dict[k]["timetable"]= tm.datetable.copy()
    res_dict[k]["shifttable"] = tm.shifttable.copy()
    
    print("trial tag:" + str(k))
    print("error num:" + str(res_dict[k]["errornum"]))
    # break

In [20]:
# res_dict[775]["shifttable"]

In [21]:
f_pkl = "./res_dict_{}.pkl".format(ts)
f_pkl

'./res_dict_20230216_183200_750595.pkl'

In [22]:
#保存
with open(f_pkl, "wb") as f:
    pickle.dump(res_dict, f)